# RICERCA A COSTO MINIMO

### Librerie utilizzate

In [1]:
import osmnx as ox
import folium
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import heapq
import time
import sys
from geopy.distance import geodesic


# LcFS - Lowest cost-first Search

In [7]:
import heapq
import time
import sys
import osmnx as ox

def lowest_cost_first_search(graph, start, goal):
    """Implementa la ricerca Lowest-Cost-First basata sul numero di incidenti, restituendo tutti i percorsi esplorati."""
    
    priority_queue = [(0, start, [start])]  # (costo cumulativo, nodo attuale, percorso)
    visited = {}  # Dizionario per memorizzare il miglior costo trovato per ciascun nodo
    explored_paths = []  # Lista dei percorsi esplorati
    
    start_time = time.time()
    memory_before = sys.getsizeof(priority_queue) + sys.getsizeof(visited) + sys.getsizeof(explored_paths)
    max_depth = 0
    
    while priority_queue:
        cost, node, path = heapq.heappop(priority_queue)  # Estrai il nodo con il minor costo
        explored_paths.append((path, cost))  # Memorizza il percorso esplorato
        max_depth = max(max_depth, len(path) - 1)
        
        if node == goal:
            execution_time = time.time() - start_time
            memory_after = sys.getsizeof(priority_queue) + sys.getsizeof(visited) + sys.getsizeof(explored_paths)
            memory_used = memory_after - memory_before
            return explored_paths, path, cost, len(explored_paths), execution_time, memory_used, max_depth
        
        if node in visited and cost >= visited[node]:
            continue  # Se il nodo è già stato visitato con un costo inferiore, lo ignoriamo
        
        visited[node] = cost  # Memorizza il costo minimo per raggiungere questo nodo
        
        for neighbor in graph.neighbors(node):
            edge_data = graph.get_edge_data(node, neighbor)
            
            # Gestione di archi multipli tra due nodi
            if isinstance(edge_data, dict):
                edge_data = min(edge_data.values(), key=lambda d: d.get("incidenti", float('inf')))
            
            edge_cost = int(edge_data.get("incidenti", 0))  # Converte il valore in intero
            new_cost = cost + edge_cost
            heapq.heappush(priority_queue, (new_cost, neighbor, path + [neighbor]))
    
    execution_time = time.time() - start_time
    memory_after = sys.getsizeof(priority_queue) + sys.getsizeof(visited) + sys.getsizeof(explored_paths)
    memory_used = memory_after - memory_before
    return explored_paths, None, float('inf'), len(explored_paths), execution_time, memory_used, max_depth 

# Caricare il grafo con il numero di incidenti sugli archi
G = ox.load_graphml("rete_bari_incidenti.graphml")

partenza = 270659688
arrivo = 1481415203

# Eseguire la ricerca Lowest-Cost-First basata sugli incidenti
explored_paths, shortest_path, min_incidents, num_explored, exec_time, memory_used, max_depth = lowest_cost_first_search(G, partenza, arrivo)

print(f"Numero totale di percorsi esplorati: {num_explored}")
print(f"Tempo di esecuzione: {exec_time:.4f} secondi")
print(f"Memoria utilizzata: {memory_used} byte")
print(f"Profondità massima raggiunta: {max_depth}")

if shortest_path:
    print(f"\nPercorso con il minor numero di incidenti trovato: {shortest_path}")
    print(f"Numero totale di incidenti lungo il percorso: {min_incidents}")
else:
    print("Nessun percorso trovato.")

#RISULTATI OTTENUTI
# Numero totale di percorsi esplorati: 6239
# Tempo di esecuzione: 0.0193 secondi
# Memoria utilizzata: 205232 byte
# Profondità massima raggiunta: 129

# Percorso con il minor numero di incidenti trovato: [270659688, 270388628, 322548994, 322548996, 12083787147, 569212252, 3860917204, 3841272949, 3860940020, 330076413, 279650482, 1483634369, 1483634284, 279655605, 280884748, 279380962, 339607009, 279655525, 10680697917, 316572227, 10680697906, 279655521, 9602691583, 1014703348, 1249973250, 4395827572, 5395447618, 5395447592, 5395448236, 5395447590, 330932109, 1459966804, 298502133, 329994613, 330003439, 1481415203]
# Numero totale di incidenti lungo il percorso: 123

Numero totale di percorsi esplorati: 6239
Tempo di esecuzione: 0.0193 secondi
Memoria utilizzata: 205232 byte
Profondità massima raggiunta: 129

Percorso con il minor numero di incidenti trovato: [270659688, 270388628, 322548994, 322548996, 12083787147, 569212252, 3860917204, 3841272949, 3860940020, 330076413, 279650482, 1483634369, 1483634284, 279655605, 280884748, 279380962, 339607009, 279655525, 10680697917, 316572227, 10680697906, 279655521, 9602691583, 1014703348, 1249973250, 4395827572, 5395447618, 5395447592, 5395448236, 5395447590, 330932109, 1459966804, 298502133, 329994613, 330003439, 1481415203]
Numero totale di incidenti lungo il percorso: 123


# MAPPATURA DEI PERCORSI 

In [8]:
def plot_paths_on_map(graph, shortest_path, explored_paths, start, goal):
    """Visualizza tutti i percorsi esplorati e il percorso più breve sulla mappa."""
    nodes = ox.graph_to_gdfs(graph, nodes=True, edges=False)
    center = (nodes.loc[start].geometry.y, nodes.loc[start].geometry.x)
    
    mappa = folium.Map(location=center, zoom_start=13)
    
    # Disegnare tutti i percorsi esplorati in blu
    for path, _ in explored_paths:
        path_coords = [(nodes.loc[node].geometry.y, nodes.loc[node].geometry.x) for node in path]
        folium.PolyLine(path_coords, color="blue", weight=2, opacity=0.5).add_to(mappa)
    
    # Disegnare il percorso più breve in rosso sopra il blu
    if shortest_path:
        path_coords = [(nodes.loc[node].geometry.y, nodes.loc[node].geometry.x) for node in shortest_path]
        folium.PolyLine(path_coords, color="red", weight=6, opacity=0.9).add_to(mappa)
    
    # Aggiungere marker per il punto di partenza e di arrivo
    folium.Marker(
        location=(nodes.loc[start].geometry.y, nodes.loc[start].geometry.x),
        popup="Partenza",
        icon=folium.Icon(color="green", icon="play")
    ).add_to(mappa)
    
    folium.Marker(
        location=(nodes.loc[goal].geometry.y, nodes.loc[goal].geometry.x),
        popup="Arrivo",
        icon=folium.Icon(color="red", icon="flag")
    ).add_to(mappa)
    
    return mappa

mappa = plot_paths_on_map(G, shortest_path, explored_paths, partenza, arrivo)
mappa.save("percorsoLcFS.html")